## Extrayendo formas de onda de P, S y ruido de estación RUS en determinado intervalo de tiempo

In [22]:
import MySQLdb
import datetime
import json
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import os

ip_fdsn = "http://10.100.100.232"
port_fdsn = "8091"
Format = "mseed"

client = Client(ip_fdsn+":"+port_fdsn)
db = MySQLdb.connect(host="10.100.100.232",    # your host, usually localhost
                     user="consulta",         # your username
                     passwd="consulta",  # your password
                     db="seiscomp3")        # name of the data base
cur = db.cursor()

cur.execute("Select  POEv.publicID,\
 Pick.waveformID_stationCode,\
Pick.phaseHint_code,Pick.phaseHint_used,Pick.evaluationMode,Pick.time_value, Pick.time_value_ms\
 from Event AS EvMF left join PublicObject AS POEv ON EvMF._oid = POEv._oid \
left join PublicObject as POOri ON EvMF.preferredOriginID=POOri.publicID \
left join Origin ON POOri._oid=Origin._oid left join PublicObject as POMag on EvMF.preferredMagnitudeID=POMag.publicID \
left join Magnitude ON Magnitude._oid = POMag._oid \
left join Arrival on Arrival._parent_oid=Origin._oid \
left join PublicObject as POOri1 on POOri1.publicID = Arrival.pickID \
left join Pick on Pick._oid= POOri1._oid \
where \
(Pick.phaseHint_used = 1 AND Pick.phaseHint_code = 'P' AND Pick.evaluationMode = 'manual' AND Pick.waveformID_stationCode = 'PTB' AND Origin.latitude_value between -4 and 14 AND Origin.longitude_value between -82 AND -67 ) \
and Origin.time_value between '2016-01-01 00:00:00' and '2016-12-31 23:59:59'")

21895L

In [23]:
#c = cur.fetchone()
#headers=[x[0] for x in cur.description] #this will extract row headers
rows = cur.fetchall()

dt=2.56
dt1 = 2*dt-0.5
dt2 = 0.5
folder = "/home/daniel/Data/wf_512_PTB/"

count = 0
p_count = 0
s_count = 0

for l in ["P", "S", "N"]:
    if not os.path.exists(folder+l):
        os.makedirs(folder+l)

for row in rows:
    t = UTCDateTime(row[-2]+datetime.timedelta(milliseconds=float(row[-1])/1000))
    
    try:
        # extrayendo forma de onda
        st = client.get_waveforms("CM", "RUS", "00", "HHZ", t-dt, t+dt)
    except:
        print row
        continue

    st.trim(t-dt, t+dt)
    st.merge(fill_value="interpolate")
    
    for tr in st:
        # guardando trazas de P y de S
        count +=1
        print count
        ch = tr.stats.channel
        name = row[2]+"_"+row[0]+"_"+t.strftime("%Y%m%d-%H%M%S")+"_"+ch+".mseed"
        print name
        tr.write(folder+row[2]+"/"+name, Format)
    
    if row[2] == "P":
        # extrayendo forma de onda de ruido de tamaño dt, 3s antes de la P
        st = client.get_waveforms("CM", "RUS", "00", "HHZ", t-4*dt, t-2*dt)
        st.trim(t-4*dt, t-2*dt)
        st.merge(fill_value="interpolate")
        
        print "len(st)=",len(st)
        for tr in st:
            p_count +=1
            count +=1
            print count
            ch = tr.stats.channel
            name = "N_"+row[0]+"_"+t.strftime("%Y%m%d-%H%M%S")+"_"+ch+".mseed"
            print name
            tr.write(folder+"N/"+name, Format)
    else:
        for tr in st: s_count += 1
        

print "Total sismogramas: %s\nSismogramas P: %s, Sismogramas S: %s"%(count, p_count, s_count)


1
P_SGC2016aaat_20160101-002341_HHZ.mseed
len(st)= 1
2
N_SGC2016aaat_20160101-002341_HHZ.mseed
3
P_SGC2016aaat_20160101-002402_HHZ.mseed
len(st)= 1
4
N_SGC2016aaat_20160101-002402_HHZ.mseed
5
P_SGC2016aabh_20160101-003954_HHZ.mseed
len(st)= 1
6
N_SGC2016aabh_20160101-003954_HHZ.mseed
7
P_SGC2016aabh_20160101-004015_HHZ.mseed
len(st)= 1
8
N_SGC2016aabh_20160101-004015_HHZ.mseed
9
P_SGC2016aacv_20160101-012601_HHZ.mseed
len(st)= 1
10
N_SGC2016aacv_20160101-012601_HHZ.mseed
11
P_SGC2016aacv_20160101-012622_HHZ.mseed
len(st)= 1
12
N_SGC2016aacv_20160101-012622_HHZ.mseed
13
P_SGC2016aago_20160101-031926_HHZ.mseed
len(st)= 1
14
N_SGC2016aago_20160101-031926_HHZ.mseed
15
P_SGC2016aago_20160101-031943_HHZ.mseed
len(st)= 1
16
N_SGC2016aago_20160101-031943_HHZ.mseed
17
P_SGC2016aahn_20160101-034824_HHZ.mseed
len(st)= 1
18
N_SGC2016aahn_20160101-034824_HHZ.mseed
19
P_SGC2016aahn_20160101-034845_HHZ.mseed
len(st)= 1
20
N_SGC2016aahn_20160101-034845_HHZ.mseed
21
P_SGC2016aaib_20160101-040456_HHZ.ms

173
P_SGC2016adnf_20160102-220515_HHZ.mseed
len(st)= 1
174
N_SGC2016adnf_20160102-220515_HHZ.mseed
175
P_SGC2016adox_20160102-225702_HHZ.mseed
len(st)= 1
176
N_SGC2016adox_20160102-225702_HHZ.mseed
177
P_SGC2016adox_20160102-225721_HHZ.mseed
len(st)= 1
178
N_SGC2016adox_20160102-225721_HHZ.mseed
179
P_SGC2016adtu_20160103-012500_HHZ.mseed
len(st)= 1
180
N_SGC2016adtu_20160103-012500_HHZ.mseed
181
P_SGC2016adtu_20160103-012520_HHZ.mseed
len(st)= 1
182
N_SGC2016adtu_20160103-012520_HHZ.mseed
183
P_SGC2016adws_20160103-025322_HHZ.mseed
len(st)= 1
184
N_SGC2016adws_20160103-025322_HHZ.mseed
185
P_SGC2016adws_20160103-025342_HHZ.mseed
len(st)= 1
186
N_SGC2016adws_20160103-025342_HHZ.mseed
187
P_SGC2016adzk_20160103-041430_HHZ.mseed
len(st)= 1
188
N_SGC2016adzk_20160103-041430_HHZ.mseed
189
P_SGC2016adzk_20160103-041451_HHZ.mseed
len(st)= 1
190
N_SGC2016adzk_20160103-041451_HHZ.mseed
191
P_SGC2016adzo_20160103-042019_HHZ.mseed
len(st)= 1
192
N_SGC2016adzo_20160103-042019_HHZ.mseed
193
P_SGC2

len(st)= 1
350
N_SGC2016agxw_20160104-185249_HHZ.mseed
351
P_SGC2016agzz_20160104-195732_HHZ.mseed
len(st)= 1
352
N_SGC2016agzz_20160104-195732_HHZ.mseed
353
P_SGC2016agzz_20160104-195752_HHZ.mseed
len(st)= 1
354
N_SGC2016agzz_20160104-195752_HHZ.mseed
355
P_SGC2016ahbg_20160104-203552_HHZ.mseed
len(st)= 1
356
N_SGC2016ahbg_20160104-203552_HHZ.mseed
357
P_SGC2016ahbg_20160104-203618_HHZ.mseed
len(st)= 1
358
N_SGC2016ahbg_20160104-203618_HHZ.mseed
359
P_SGC2016ahch_20160104-210623_HHZ.mseed
len(st)= 1
360
N_SGC2016ahch_20160104-210623_HHZ.mseed
361
P_SGC2016ahch_20160104-210644_HHZ.mseed
len(st)= 1
362
N_SGC2016ahch_20160104-210644_HHZ.mseed
363
P_SGC2016ahcs_20160104-211954_HHZ.mseed
len(st)= 1
364
N_SGC2016ahcs_20160104-211954_HHZ.mseed
365
P_SGC2016ahcs_20160104-212014_HHZ.mseed
len(st)= 1
366
N_SGC2016ahcs_20160104-212014_HHZ.mseed
367
P_SGC2016ahcx_20160104-212510_HHZ.mseed
len(st)= 1
368
N_SGC2016ahcx_20160104-212510_HHZ.mseed
369
P_SGC2016ahcx_20160104-212532_HHZ.mseed
len(st)= 1

526
N_SGC2016akbz_20160106-122237_HHZ.mseed
527
P_SGC2016akcp_20160106-124050_HHZ.mseed
len(st)= 1
528
N_SGC2016akcp_20160106-124050_HHZ.mseed
529
P_SGC2016akcp_20160106-124111_HHZ.mseed
len(st)= 1
530
N_SGC2016akcp_20160106-124111_HHZ.mseed
531
P_SGC2016akjl_20160106-160840_HHZ.mseed
len(st)= 1
532
N_SGC2016akjl_20160106-160840_HHZ.mseed
533
P_SGC2016akjl_20160106-160901_HHZ.mseed
len(st)= 1
534
N_SGC2016akjl_20160106-160901_HHZ.mseed
535
P_SGC2016aknc_20160106-175843_HHZ.mseed
len(st)= 1
536
N_SGC2016aknc_20160106-175843_HHZ.mseed
537
P_SGC2016aknc_20160106-175904_HHZ.mseed
len(st)= 1
538
N_SGC2016aknc_20160106-175904_HHZ.mseed
539
P_SGC2016akro_20160106-201424_HHZ.mseed
len(st)= 1
540
N_SGC2016akro_20160106-201424_HHZ.mseed
541
P_SGC2016akro_20160106-201446_HHZ.mseed
len(st)= 1
542
N_SGC2016akro_20160106-201446_HHZ.mseed
543
P_SGC2016akrt_20160106-202033_HHZ.mseed
len(st)= 1
544
N_SGC2016akrt_20160106-202033_HHZ.mseed
545
P_SGC2016aktf_20160106-210358_HHZ.mseed
len(st)= 1
546
N_SGC2

701
P_SGC2016aosc_20160109-010349_HHZ.mseed
len(st)= 1
702
N_SGC2016aosc_20160109-010349_HHZ.mseed
703
P_SGC2016aosc_20160109-010410_HHZ.mseed
len(st)= 1
704
N_SGC2016aosc_20160109-010410_HHZ.mseed
705
P_SGC2016aovo_20160109-024737_HHZ.mseed
len(st)= 1
706
N_SGC2016aovo_20160109-024737_HHZ.mseed
707
P_SGC2016aovo_20160109-024758_HHZ.mseed
len(st)= 1
708
N_SGC2016aovo_20160109-024758_HHZ.mseed
709
P_SGC2016aowr_20160109-032219_HHZ.mseed
len(st)= 1
710
N_SGC2016aowr_20160109-032219_HHZ.mseed
711
P_SGC2016aowr_20160109-032240_HHZ.mseed
len(st)= 1
712
N_SGC2016aowr_20160109-032240_HHZ.mseed
713
P_SGC2016aoxp_20160109-035014_HHZ.mseed
len(st)= 1
714
N_SGC2016aoxp_20160109-035014_HHZ.mseed
715
P_SGC2016aoxp_20160109-035053_HHZ.mseed
len(st)= 1
716
N_SGC2016aoxp_20160109-035053_HHZ.mseed
717
P_SGC2016aoye_20160109-040735_HHZ.mseed
len(st)= 1
718
N_SGC2016aoye_20160109-040735_HHZ.mseed
719
P_SGC2016aoye_20160109-040755_HHZ.mseed
len(st)= 1
720
N_SGC2016aoye_20160109-040755_HHZ.mseed
721
P_SGC2

...

38413
P_SGC2016zkno_20161227-123655_HHZ.mseed
len(st)= 1
38414
N_SGC2016zkno_20161227-123655_HHZ.mseed
38415
P_SGC2016zkoi_20161227-125945_HHZ.mseed
len(st)= 1
38416
N_SGC2016zkoi_20161227-125945_HHZ.mseed
38417
P_SGC2016zkoi_20161227-130005_HHZ.mseed
len(st)= 1
38418
N_SGC2016zkoi_20161227-130005_HHZ.mseed
38419
P_SGC2016zkpp_20161227-133801_HHZ.mseed
len(st)= 1
38420
N_SGC2016zkpp_20161227-133801_HHZ.mseed
38421
P_SGC2016zkpp_20161227-133820_HHZ.mseed
len(st)= 1
38422
N_SGC2016zkpp_20161227-133820_HHZ.mseed
38423
P_SGC2016zkql_20161227-140410_HHZ.mseed
len(st)= 1
38424
N_SGC2016zkql_20161227-140410_HHZ.mseed
38425
P_SGC2016zkql_20161227-140430_HHZ.mseed
len(st)= 1
38426
N_SGC2016zkql_20161227-140430_HHZ.mseed
38427
P_SGC2016zksr_20161227-151215_HHZ.mseed
len(st)= 1
38428
N_SGC2016zksr_20161227-151215_HHZ.mseed
38429
P_SGC2016zksr_20161227-151230_HHZ.mseed
len(st)= 1
38430
N_SGC2016zksr_20161227-151230_HHZ.mseed
38431
P_SGC2016zkzu_20161227-184730_HHZ.mseed
len(st)= 1
38432
N_SGC2016z

38573
P_SGC2016zmfs_20161228-105500_HHZ.mseed
len(st)= 1
38574
N_SGC2016zmfs_20161228-105500_HHZ.mseed
38575
P_SGC2016zmfs_20161228-105520_HHZ.mseed
len(st)= 1
38576
N_SGC2016zmfs_20161228-105520_HHZ.mseed
38577
P_SGC2016zmhq_20161228-115337_HHZ.mseed
len(st)= 1
38578
N_SGC2016zmhq_20161228-115337_HHZ.mseed
38579
P_SGC2016zmhq_20161228-115406_HHZ.mseed
len(st)= 1
38580
N_SGC2016zmhq_20161228-115406_HHZ.mseed
38581
P_SGC2016zmii_20161228-121504_HHZ.mseed
len(st)= 1
38582
N_SGC2016zmii_20161228-121504_HHZ.mseed
38583
P_SGC2016zmii_20161228-121520_HHZ.mseed
len(st)= 1
38584
N_SGC2016zmii_20161228-121520_HHZ.mseed
38585
P_SGC2016zmjb_20161228-123610_HHZ.mseed
len(st)= 1
38586
N_SGC2016zmjb_20161228-123610_HHZ.mseed
38587
P_SGC2016zmjb_20161228-123632_HHZ.mseed
len(st)= 1
38588
N_SGC2016zmjb_20161228-123632_HHZ.mseed
38589
P_SGC2016zmmk_20161228-141836_HHZ.mseed
len(st)= 1
38590
N_SGC2016zmmk_20161228-141836_HHZ.mseed
38591
P_SGC2016zmmk_20161228-141857_HHZ.mseed
len(st)= 1
38592
N_SGC2016z

len(st)= 1
38736
N_SGC2016zobj_20161229-105937_HHZ.mseed
38737
P_SGC2016zobj_20161229-105956_HHZ.mseed
len(st)= 1
38738
N_SGC2016zobj_20161229-105956_HHZ.mseed
38739
P_SGC2016zodv_20161229-121346_HHZ.mseed
len(st)= 1
38740
N_SGC2016zodv_20161229-121346_HHZ.mseed
38741
P_SGC2016zodv_20161229-121401_HHZ.mseed
len(st)= 1
38742
N_SGC2016zodv_20161229-121401_HHZ.mseed
38743
P_SGC2016zoef_20161229-122623_HHZ.mseed
len(st)= 1
38744
N_SGC2016zoef_20161229-122623_HHZ.mseed
38745
P_SGC2016zoef_20161229-122634_HHZ.mseed
len(st)= 1
38746
N_SGC2016zoef_20161229-122634_HHZ.mseed
38747
P_SGC2016zofh_20161229-125836_HHZ.mseed
len(st)= 1
38748
N_SGC2016zofh_20161229-125836_HHZ.mseed
38749
P_SGC2016zofh_20161229-125855_HHZ.mseed
len(st)= 1
38750
N_SGC2016zofh_20161229-125855_HHZ.mseed
38751
P_SGC2016zogy_20161229-134906_HHZ.mseed
len(st)= 1
38752
N_SGC2016zogy_20161229-134906_HHZ.mseed
38753
P_SGC2016zogy_20161229-134922_HHZ.mseed
len(st)= 1
38754
N_SGC2016zogy_20161229-134922_HHZ.mseed
38755
P_SGC2016z

len(st)= 1
38906
N_SGC2016zprh_20161230-081137_HHZ.mseed
38907
P_SGC2016zprj_20161230-081310_HHZ.mseed
len(st)= 1
38908
N_SGC2016zprj_20161230-081310_HHZ.mseed
38909
P_SGC2016zprj_20161230-081332_HHZ.mseed
len(st)= 1
38910
N_SGC2016zprj_20161230-081332_HHZ.mseed
38911
P_SGC2016zpry_20161230-083010_HHZ.mseed
len(st)= 1
38912
N_SGC2016zpry_20161230-083010_HHZ.mseed
38913
P_SGC2016zpry_20161230-083031_HHZ.mseed
len(st)= 1
38914
N_SGC2016zpry_20161230-083031_HHZ.mseed
38915
P_SGC2016zpsl_20161230-084614_HHZ.mseed
len(st)= 1
38916
N_SGC2016zpsl_20161230-084614_HHZ.mseed
38917
P_SGC2016zpsl_20161230-084642_HHZ.mseed
len(st)= 1
38918
N_SGC2016zpsl_20161230-084642_HHZ.mseed
38919
P_SGC2016zpss_20161230-085424_HHZ.mseed
len(st)= 1
38920
N_SGC2016zpss_20161230-085424_HHZ.mseed
38921
P_SGC2016zptv_20161230-092801_HHZ.mseed
len(st)= 1
38922
N_SGC2016zptv_20161230-092801_HHZ.mseed
38923
P_SGC2016zptv_20161230-092829_HHZ.mseed
len(st)= 1
38924
N_SGC2016zptv_20161230-092829_HHZ.mseed
38925
P_SGC2016z

len(st)= 1
39066
N_SGC2016zrlp_20161231-073452_HHZ.mseed
39067
P_SGC2016zrmx_20161231-081435_HHZ.mseed
len(st)= 1
39068
N_SGC2016zrmx_20161231-081435_HHZ.mseed
39069
P_SGC2016zrmx_20161231-081451_HHZ.mseed
len(st)= 1
39070
N_SGC2016zrmx_20161231-081451_HHZ.mseed
39071
P_SGC2016zroz_20161231-091649_HHZ.mseed
len(st)= 1
39072
N_SGC2016zroz_20161231-091649_HHZ.mseed
39073
P_SGC2016zroz_20161231-091716_HHZ.mseed
len(st)= 1
39074
N_SGC2016zroz_20161231-091716_HHZ.mseed
39075
P_SGC2016zrpg_20161231-092450_HHZ.mseed
len(st)= 1
39076
N_SGC2016zrpg_20161231-092450_HHZ.mseed
39077
P_SGC2016zrpg_20161231-092510_HHZ.mseed
len(st)= 1
39078
N_SGC2016zrpg_20161231-092510_HHZ.mseed
39079
P_SGC2016zrqr_20161231-100831_HHZ.mseed
len(st)= 1
39080
N_SGC2016zrqr_20161231-100831_HHZ.mseed
39081
P_SGC2016zrqr_20161231-100852_HHZ.mseed
len(st)= 1
39082
N_SGC2016zrqr_20161231-100852_HHZ.mseed
39083
P_SGC2016zrro_20161231-103538_HHZ.mseed
len(st)= 1
39084
N_SGC2016zrro_20161231-103538_HHZ.mseed
39085
P_SGC2016z

In [19]:
c= rows[1]
dt =1
t = UTCDateTime(c[-2]+datetime.timedelta(milliseconds=float(c[-1])/1000))

# extrayendo forma de onda
st = client.get_waveforms("CM", "RUS", "00", "HH*", t-dt, t+dt)
st.trim(t-dt, t+dt)
st.merge(fill_value="interpolate")

print t.strftime("%Y%m%d-%H%M%S")
print st

20160101-002357
3 Trace(s) in Stream:
CM.RUS.00.HHZ | 2016-01-01T00:23:56.588393Z - 2016-01-01T00:23:58.588393Z | 100.0 Hz, 201 samples
CM.RUS.00.HHE | 2016-01-01T00:23:56.588393Z - 2016-01-01T00:23:58.588393Z | 100.0 Hz, 201 samples
CM.RUS.00.HHN | 2016-01-01T00:23:56.588393Z - 2016-01-01T00:23:58.588393Z | 100.0 Hz, 201 samples


In [20]:
dt

1

In [21]:
cur.description 

(('publicID', 253, 11, 255, 255, 0, 1),
 ('waveformID_stationCode', 254, 3, 8, 8, 0, 1),
 ('phaseHint_code', 254, 1, 32, 32, 0, 1),
 ('phaseHint_used', 1, 1, 1, 1, 0, 1),
 ('evaluationMode', 253, 6, 64, 64, 0, 1),
 ('time_value', 12, 19, 19, 19, 0, 1),
 ('time_value_ms', 3, 6, 11, 11, 0, 1))